In [163]:
import numpy as np
import random
import pandas as pd
from sklearn.linear_model import LinearRegression
from copy import deepcopy

In [164]:
def generate_graph(num_nodes):
    # Generate a random adjacency matrix representing the graph connectivity
    graph_matrix = np.random.randint(0, 2, size=(num_nodes, num_nodes))
    np.fill_diagonal(graph_matrix, 0)  # Ensure no self-loops

    return graph_matrix

#def generate_time_dependent_weights(num_nodes, num_intervals):
#    # Generate random time-dependent weights for edges
#    time_dependent_weights = np.random.rand(num_nodes, num_nodes, num_intervals) * 10
#
#    return time_dependent_weights

def generate_time_dependent_weights(num_nodes, num_intervals):
        # Generate random time-dependent weights for edges
        time_dependent_weights = np.random.rand(num_nodes, num_nodes, num_intervals) * 10

        # Scale the entire weight matrix to increase the range of weights
        #weight_scale_factor = 5  # Adjust this factor as needed
        weight_scale_factor = 30
        time_dependent_weights *= weight_scale_factor

        return time_dependent_weights

def generate_start_times(num_intervals):
    # Generate random start times within the given time intervals
    return np.random.randint(0, num_intervals, size=num_intervals)

def generate_sample_data(num_nodes, num_intervals):
    graph_matrix = generate_graph(num_nodes)
    time_dependent_weights = generate_time_dependent_weights(num_nodes, num_intervals)
    start_times = generate_start_times(num_intervals)

    return graph_matrix, time_dependent_weights, start_times

In [165]:
graph_matrix, td_weights, start_times = generate_sample_data(20,24)
graph_matrix, td_weights, start_times

(array([[0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0],
        [1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1],
        [0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1],
        [0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1],
        [1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1],
        [1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0],
        [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1],
        [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
        [1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
        [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1],
        [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
        [0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
        [1, 0, 0, 0,

In [166]:
class Path:
    def __init__(self, num_nodes, num_intervals, graph_matrix, td_weights, start_times, start_node, end_node):
        self.num_nodes = num_nodes
        self.num_intervals = num_intervals
        self.graph_matrix = graph_matrix
        self.td_weights = td_weights
        self.start_times = start_times
        self.start_node = start_node
        self.end_node = end_node

        # Randomly initialize a solution
        self.solution = self.generate_valid_solution()
        self.value = self.calculate_solution_value()

    def generate_valid_solution(self):
        #valid_solution = [np.random.randint(0, self.num_nodes)]
        valid_solution = [self.start_node]
        
        i = 1
        #for i in range(1, self.num_intervals + 1):
        while i < self.num_intervals + 1:
            if i == self.num_intervals:
                i = 1
                continue
            
            current_node = valid_solution[-1]
            
            # Find valid edges for the current node
            valid_edges = np.where(self.graph_matrix[current_node] == 1)[0]
            
            # Ensure there's at least one valid edge
            if len(valid_edges) > 0:
                # Randomly choose a valid edge
                next_node = np.random.choice(valid_edges)
                valid_solution.append(next_node)
                
                if next_node == self.end_node:
                    break
            else:
                # If no valid edges, stay in the current node
                #ovo ne valja ovako, mora da se popravi
                valid_solution.append(current_node)
            i =+ 1    

        return np.array(valid_solution)
        
    
    def calculate_solution_value(self):
        total_value = 0
        n = len(self.td_weights)
        #for i in range(self.num_intervals - 1):
        for i in range(len(self.solution) - 1):
            # Calculate the value of the solution based on time-dependent weights
            current_node = self.solution[i % n]
            next_node = self.solution[(i + 1) % n]
            start_time = self.start_times[i % n]

            # Add the time-dependent weight for the edge between current_node and next_node at the given start_time
            total_value += self.td_weights[current_node, next_node, start_time]

        return total_value
         

In [167]:
path = Path(10, 24, graph_matrix, td_weights, start_times, 2, 18)
print(path.solution)
print(path.value)

[ 2  7 11 19  5  2  4 18]
1209.64250266424


In [168]:
 def calculate_solution_value(solution, graph_matrix, td_weights, start_times):
        total_value = 0
        n = len(td_weights)
        #for i in range(self.num_intervals - 1):
        for i in range(len(solution) - 1):
            # Calculate the value of the solution based on time-dependent weights
            current_node = solution[i % n]
            next_node = solution[(i + 1) % n]
            start_time = start_times[i % n]

            # Add the time-dependent weight for the edge between current_node and next_node at the given start_time
            total_value += td_weights[current_node, next_node, start_time]
        #print(total_value)

        return total_value

In [169]:
def generate_valid_solution(graph_matrix, start_node, end_node):
        #valid_solution = [np.random.randint(0, self.num_nodes)]
        valid_solution = [start_node]
        
        while True:
            current_node = valid_solution[-1]
            
            # Find valid edges for the current node
            valid_edges = np.where(graph_matrix[current_node] == 1)[0]
            
            # Ensure there's at least one valid edge
            if len(valid_edges) > 0:
                # Randomly choose a valid edge
                next_node = np.random.choice(valid_edges)
                valid_solution.append(next_node)
                
                if next_node == end_node:
                    break
            else:
                # If no valid edges, stay in the current node
                #ovo ne valja ovako, mora da se popravi
                valid_solution.append(current_node)    

        return np.array(valid_solution)

In [170]:
def make_small_change(original_solution, graph_matrix, start_node, end_node):
    #num_intervals = len(original_solution)
    #new_solution = deepcopy(original_solution)

    # Choose a random time interval to make a small change
    #interval_to_change = np.random.randint(0, num_intervals)
    
    # Ensure we don't change the start or end node
    #while interval_to_change == 0 or interval_to_change == num_intervals - 1:
    #    interval_to_change = np.random.randint(0, num_intervals)

    # Choose a random node as the destination
    #new_node = np.random.randint(0, len(graph_matrix))

    # Ensure the new node is a valid choice
    #valid_edges = np.where(graph_matrix[new_solution[interval_to_change - 1]] == 1)[0]
    #while len(valid_edges) == 0 or new_node == start_node or new_node == end_node:
    #    new_node = np.random.randint(0, len(graph_matrix))
    #    valid_edges = np.where(graph_matrix[new_solution[interval_to_change - 1]] == 1)[0]

    # Update the solution
    #new_solution[interval_to_change] = new_node

    #return new_solution

    num_intervals = len(original_solution)
    new_solution = deepcopy(original_solution)

    # Randomly decide whether to make the original change or generate a new solution
    if np.random.rand() < 0.5:
        print('modify')
        # Original change: choose a random time interval and a new node
        interval_to_change = np.random.randint(0, num_intervals)
        
        # Ensure we don't change the start or end node
        while interval_to_change == 0 or interval_to_change == num_intervals - 1:
            interval_to_change = np.random.randint(0, num_intervals)

        new_node = np.random.randint(0, len(graph_matrix))

        # Ensure the new node is a valid choice
        valid_edges = np.where(graph_matrix[new_solution[interval_to_change - 1]] == 1)[0]
        while len(valid_edges) == 0 or new_node == start_node or new_node == end_node:
            new_node = np.random.randint(0, len(graph_matrix))
            valid_edges = np.where(graph_matrix[new_solution[interval_to_change - 1]] == 1)[0]

        # Update the solution
        new_solution[interval_to_change] = new_node

    else:
        #ovo izgleda ne radi kako treba, trbealo bi da izaberemo random indeks, onda napravimo novo resenje gde je taj indeks pocetni node a end node je
        #isti
        print('append')
        new_solution = []
        #new_start = interval_to_change = np.random.randint(0, len(graph_matrix))
        indeks = np.random.randint(0, len(original_solution))
        new_start = original_solution[indeks]
        print(new_start)
        for i in range(0, indeks):
            #print('i: ', i)
            new_solution.append(original_solution[i])
            #print(new_solution)
        continuation = generate_valid_solution(graph_matrix, new_start, end_node)
        n = len(continuation)
        for i in range(0, n):
            new_solution.append(continuation[i])
            #print(new_solution)
        
        
        # Generate a new solution with the start node being the randomly picked node
        #new_start_node = np.random.randint(0, len(graph_matrix))
        
        # Ensure the new start node is a valid choice
        #valid_start_edges = np.where(graph_matrix[end_node] == 1)[0]
        #while len(valid_start_edges) == 0 or new_start_node == end_node:
        #    new_start_node = np.random.randint(0, len(graph_matrix))
        #    valid_start_edges = np.where(graph_matrix[end_node] == 1)[0]

        # Generate a new solution with the new start node
        #new_solution = [new_start_node]

        #for _ in range(1, num_intervals - 1):
        #    current_node = new_solution[-1]

            # Find valid edges for the current node
        #    valid_edges = np.where(graph_matrix[current_node] == 1)[0]

            # Ensure there's at least one valid edge
        #    if len(valid_edges) > 0:
                # Randomly choose a valid edge
        #        next_node = np.random.choice(valid_edges)
        #        new_solution.append(next_node)

                # Check if the next node is the original end node
        #        if next_node == end_node:
        #            break
        #    else:
                # If no valid edges, stay in the current node
        #        new_solution.append(current_node)

    return new_solution

    

In [171]:
def local_search(graph_matrix, td_weights, start_times, num_iters, start_node, end_node):
    path = Path(20, 24, graph_matrix, td_weights, start_times, start_node, end_node)
    solution = path.solution
    value = path.value
    best_solution = deepcopy(solution)
    best_value = value

    for i in range(num_iters):
        new_solution = make_small_change(solution, graph_matrix, start_node, end_node)
        new_value = calculate_solution_value(new_solution, graph_matrix, td_weights, start_times)
        
        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)
            if value < best_value:
                best_value = value
                best_solution = deepcopy(solution)
        print(solution, value, i)

    return best_solution, best_value

In [172]:
local_search(graph_matrix, td_weights, start_times, 100, 2, 18)

modify
[ 2 19  2  7  3 13  9 14 18] 1316.5977188346972 0
modify
[ 2 19  2  7  3 13  9  0 18] 1259.9269734950524 1
append
7
[ 2 19  2  7  3 13  9  0 18] 1259.9269734950524 2
modify
[ 2 19  2  7  3 13  9  0 18] 1259.9269734950524 3
append
2
[ 2 19  2  7  3 13  9  0 18] 1259.9269734950524 4
modify
[ 2 19  2  7  3 13  9  0 18] 1259.9269734950524 5
append
13
[ 2 19  2  7  3 13  9  0 18] 1259.9269734950524 6
append
3
[ 2 19  2  7  3 13  9  0 18] 1259.9269734950524 7
modify
[ 2 19 19  7  3 13  9  0 18] 1175.971541926972 8
append
9
[ 2 19 19  7  3 13  9  0 18] 1175.971541926972 9
modify
[ 2 19 19  7  3 13  9  0 18] 1175.971541926972 10
modify
[ 2 19 19  7  3 13  9  0 18] 1175.971541926972 11
modify
[ 2 19 19  1  3 13  9  0 18] 1031.7022111991678 12
modify
[ 2 19 19  1  3 13  9  0 18] 1031.7022111991678 13
modify
[ 2 11 19  1  3 13  9  0 18] 845.9867472149856 14
modify
[ 2 11 19  1  3 13  9  0 18] 845.9867472149856 15
append
11
[2, 11, 19, 15, 19, 18] 557.1062438641211 16
modify
[2, 11, 19, 15,

([2, 11, 18], 185.32335762422753)

In [173]:
def simulated_annealing(graph_matrix, td_weights, start_times, num_iters, start_node, end_node):
    path = Path(20, 24, graph_matrix, td_weights, start_times, start_node, end_node)
    solution = path.solution
    value = path.value
    best_solution = deepcopy(solution)
    best_value = value

    for i in range(1, num_iters + 1):
        new_solution = make_small_change(solution, graph_matrix, start_node, end_node)
        new_value = calculate_solution_value(new_solution, graph_matrix, td_weights, start_times)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)
            if value < best_value:
                best_value = value
                best_solution = deepcopy(solution)

        elif random.random() < 1/i:
            value = new_value
            solution = deepcopy(new_solution)
        print(solution, value, i)    

    return best_solution, best_value

In [174]:
simulated_annealing(graph_matrix, td_weights, start_times, 100, 2, 18)

modify
[ 2 12  4 17  0  3  2 13  1  9 18] 1392.328686501534 1
append
2
[ 2 12  4 17  0  3  2 13  1  9 18] 1392.328686501534 2
append
4
[2, 12, 4, 18] 457.2484100751758 3
append
4
[2, 12, 4, 18] 457.2484100751758 4
append
4
[2, 12, 4, 14, 19, 18] 581.9986827226455 5
modify
[2, 12, 4, 14, 19, 18] 581.9986827226455 6
append
4
[2, 12, 4, 14, 19, 18] 581.9986827226455 7
append
14
[2, 12, 4, 14, 18] 350.16992803664385 8
append
12
[2, 12, 15, 2, 4, 10, 15, 10, 8, 14, 18] 1441.8930085507805 9
modify
[2, 12, 15, 2, 4, 10, 15, 10, 8, 14, 18] 1441.8930085507805 10
append
15
[2, 12, 15, 2, 4, 10, 15, 10, 8, 14, 18] 1441.8930085507805 11
modify
[2, 12, 15, 2, 4, 10, 15, 10, 8, 8, 18] 1248.2207446959885 12
append
2
[2, 12, 15, 2, 4, 10, 15, 10, 8, 8, 18] 1248.2207446959885 13
append
12
[2, 12, 1, 7, 12, 19, 18] 588.3941552283898 14
modify
[2, 12, 1, 7, 12, 19, 18] 588.3941552283898 15
modify
[2, 12, 1, 7, 12, 19, 18] 588.3941552283898 16
modify
[2, 12, 1, 7, 12, 19, 18] 588.3941552283898 17
append
1

([2, 11, 18], 185.32335762422753)

In [175]:
def shaking(original_solution, k, graph_matrix, start_node, end_node):
    ids = random.sample(range(len(original_solution)), k)
    new_solution = deepcopy(original_solution)
    for id in ids:
        #aj nastavi
        

SyntaxError: incomplete input (1266951932.py, line 6)

In [ ]:
def vns(graph_matrix, td_weights, start_times, num_iters, start_node, end_node, k_min, k_max):
    path = Path(20, 24, graph_matrix, td_weights, start_times, start_node, end_node)
    solution = path.solution
    value = path.value
    best_solution = deepcopy(solution)
    best_value = value

    for i in range(0, num_iters):
        for k in range(k_min, k_max):
            new_solution = shaking(solution, k, graph_matrix, start_node, end_node)
            new_value = calculate_solution_value(new_solution, graph_matrix, td_weights, start_times)
            print(value)

            if new_value < value:
                value = new_value
                solution = deepcopy(new_solution)
                if value < best_value:
                    best_value = value
                    best_solution = deepcopy(solution)

            elif random.random() < 1/i:
                value = new_value
                solution = deepcopy(new_solution)

        return best_solution, best_value